# Supervised Learning Final Project

Diabetes is a chronic condition that occurs when the pancreas can no longer make insulin, or the body cannot effectively use insulin. Insulin is a hormone created in the pancreas that to move glucose into cells in the body. Glucose is a source of energy for the body. A person with diabetes is unable to control the levels of glucose in their blood. High levels of glucose over a long term is associated with organ failure and other damage to the body. As per [this report](https://idf.org/media/uploads/2024/06/IDF-Annual-Report-2023.pdf) from the International Diabetes Federation, an estimated 540 million people live with diabetes.

Early diabetes detection and prediction is an important tool to prevent diabetes. While there are several modern tests/features that can be used for diabetes detection, we will use some of the historically common ones that are cheaper and readily available. e.g. BMI, Glucose, Insulin, Age etc.

New research has shown that data preprocessed with Recursive Feature Elimination(RFE) and methods like eXtreme Gradient Boosting(XGBoost) and Stacking can provide a higher performance compared to Random Forest classifiers. The goal of these project is to explore the improvement in accuracy with these methods.

In [ ]:
%pip install pandas numpy matplotlib seaborn scikit-learn xgboost kagglehub docutils

import kagglehub
import docutils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from kagglehub import KaggleDatasetAdapter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from xgboost import XGBClassifier

## Data

For this project, we will use the Pima Indians Diabetes Dataset made available by UCI Machine Learning on Kaggle. This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage. 

In [ ]:
# Download latest version
data = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "uciml/pima-indians-diabetes-database",
  "diabetes.csv",
)
print(data.head())
print(data.info())

## Data Cleaning
The small number of records and the constraints mentioned in the previous section can skew the data and affect the predictions.

In [ ]:
# Find distribution of target variable
data.hist(figsize = (10,10))
plt.show()

### Handling zero values
As shown on the graph, several features in the dataset have an impossible zero value. e.g. Glucose, Blood Pressure and BMI. These seem to be missing values that need to be imputed. We will impute these values using the median value of these columns

In [ ]:
# Replace zero values with median
zero_columns = ['Glucose', 'BloodPressure', 'BMI', 'SkinThickness', 'Insulin']
for col in zero_columns:
    data[col] = data[col].replace(0, data[col].median())

print(data.head())

In [ ]:
# Histplot with cleaned data
data.hist(figsize = (10,10))
plt.show()

## Exploratory Data Analysis

In [ ]:
sns.pairplot(data, hue='Outcome', diag_kind='hist', height=1.5)
plt.show()

### Pairplot insights
- A counterintuitive finding is that the Glucose and Insulin are not linearly related.
- The pairplot also shows that BMI and Skin Thickness are correlated and therefore only one of them might be necessary for prediction

In [ ]:
corr_matrix = data.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Heatmap')
plt.show()

### Correlation Heatmap insights
- Outcomes are highly correlated to Glucose, BMI and Age. Intuitively, these should be the top three predictors for the models.
- Insulin, Blood Pressure and DiabetesPedigreeFunction are poor predictors becuase of their low correlation

## Models

Before applying the models, we need to scale the data to ensure that the features are on the same scale before they are selected using a Feature Selector.

In [ ]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Recursive Feature Elimination using Logistic Regression as the base estimator
We will now use the RFE available in scikit-learn to select 4 features using recursive feature elimination. The features that provide the highest accuracy will be chosen. The estimator used is Logistic Regression. We will then create a training and test sample that is derived based on these features

In [ ]:
lr_model = LogisticRegression(random_state=42)
rfe = RFE(estimator=lr_model, n_features_to_select=4, step=1)
rfe.fit(X_train_scaled, y_train)

# Get the selected features
selected_features = X.columns[rfe.support_]
print("Selected Features by RFE:", selected_features)

# Create new datasets with only the selected features
X_train_rfe = rfe.transform(X_train_scaled)
X_test_rfe = rfe.transform(X_test_scaled)


### Random Forest without Feature Selection


In [ ]:
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5, scoring='accuracy')
rf_grid.fit(X_train_scaled, y_train)
rf_best = rf_grid.best_estimator_

print("\nBest Random Forest Parameters (without feature selection):", rf_grid.best_params_)
print("Random Forest Performance (without feature selection):")
print(classification_report(y_test, rf_best.predict(X_test_scaled)))
print("Accuracy Score:", accuracy_score(y_test, rf_best.predict(X_test_scaled)))

### Random Forest with feature selection

In [ ]:
rf_sfs_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5, scoring='accuracy')
rf_sfs_grid.fit(X_train_sfs, y_train)
rf_sfs_best = rf_sfs_grid.best_estimator_

print("\nBest Random Forest Parameters (with feature selection):", rf_sfs_grid.best_params_)
print("Random Forest Performance (with feature selection):")
print(classification_report(y_test, rf_sfs_best.predict(X_test_sfs)))
print("Accuracy Score:", accuracy_score(y_test, rf_sfs_best.predict(X_test_sfs)))

### XGBoost
XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that is widely used to solve many data science problems in a fast and accurate way. 

In [ ]:
xgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3]
}
xgb_grid = GridSearchCV(XGBClassifier(random_state=42), xgb_params, cv=5, scoring='accuracy')
xgb_grid.fit(X_train_scaled, y_train)
xgb_best = xgb_grid.best_estimator_

print("\nBest XGBoost Parameters:", xgb_grid.best_params_)
print("XGBoost Performance:")
print(classification_report(y_test, xgb_best.predict(X_test_scaled)))
print("Accuracy Score:", accuracy_score(y_test, xgb_best.predict(X_test_scaled)))

### Stacking
Stacking is an ensemble machine learning method that can classify data with results from different classifiers and a training result within itself. The model emerged by the results from different models and the result of training within itself is called the meta-model.

In [ ]:
svm_model = SVC(C=10, gamma=0.01, kernel='rbf', probability=True, random_state=42)
rf_model = RandomForestClassifier(max_depth=5, min_samples_split=8, n_estimators=108, random_state=42)
xgb_model = XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=3, min_child_weight=10, random_state=42)

# Apply feature selection to each base model
sfs_svm = SequentialFeatureSelector(svm_model, n_features_to_select=5, direction='forward', cv=5)
sfs_rf = SequentialFeatureSelector(rf_model, n_features_to_select=5, direction='forward', cv=5)
sfs_xgb = SequentialFeatureSelector(xgb_model, n_features_to_select=5, direction='forward', cv=5)

# Fit feature selectors and transform the data
X_train_sfs_svm = sfs_svm.fit_transform(X_train_scaled, y_train)
X_train_sfs_rf = sfs_rf.fit_transform(X_train_scaled, y_train)
X_train_sfs_xgb = sfs_xgb.fit_transform(X_train_scaled, y_train)

# Define base models for stacking with feature selection
base_models = [
    ('svm', Pipeline([('selector', sfs_svm), ('model', svm_model)])),
    ('rf', Pipeline([('selector', sfs_rf), ('model', rf_model)])),
    ('xgb', Pipeline([('selector', sfs_xgb), ('model', xgb_model)]))
]

# Define meta-model (using Logistic Regression as the final estimator)
meta_model = LogisticRegression(random_state=42)

# Create the stacking classifier
stacking_classifier = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method='predict_proba'
)

# Fit the stacking classifier
stacking_classifier.fit(X_train_scaled, y_train)

# Evaluate the stacking model
print("\nStacking Model Performance:")
y_pred_stacking = stacking_classifier.predict(X_test_scaled)
print(classification_report(y_test, y_pred_stacking))
stacking_accuracy = accuracy_score(y_test, y_pred_stacking)
print(f"Accuracy Score: {stacking_accuracy:.2f}%")

## Results and Analysis

### ROC Curve Comparison
The ROC curve across models shows that Random Forest with no feature selection has the highest AUC at 0.84 while Stacking RF has relatively the poorest AUC at 0.77

In [ ]:
models = [rf_best, rf_sfs_best, xgb_best, stacking_model]
model_names = ['Random Forest (No FS)', 'Random Forest (With FS)', 'XGBoost', 'Stacking RF']

plt.figure(figsize=(10, 6))
for model, name in zip(models, model_names):
    if name == 'Random Forest (With FS)':
        y_pred_proba = model.predict_proba(X_test_sfs)[:, 1]
    else:
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC={auc:.2f})')

plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend()
plt.show()

### Confusion Matrices
The confusion matrix shows that Random Forest with Feature Selection and XGBoost perform the best at identifying True Positives and True Negatives but struggle with False Negatives and False Positives

In [ ]:
def plot_confusion_matrix(model, X_test, y_test, title):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    disp.plot(cmap='Blues')
    plt.title(title)
    plt.show()

for model, name in zip(models, model_names):
    if name == 'Random Forest (With FS)':
        plot_confusion_matrix(model, X_test_sfs, y_test, f"Confusion Matrix - {name}")
    else:
        plot_confusion_matrix(model, X_test_scaled, y_test, f"Confusion Matrix - {name}")

### Model Accuracy comparison
The model accuracy comparison shows that stacking is clearly the most powerful model but the confusion matrix and the low AUC score shows that the model is struggling to differentiate between True and False Positives.These results can be explained by the outcome imbalance in the original dataset

In [ ]:
accuracies = [
    accuracy_score(y_test, rf_best.predict(X_test_scaled)) * 100,
    accuracy_score(y_test, rf_sfs_best.predict(X_test_sfs)) * 100,
    accuracy_score(y_test, xgb_best.predict(X_test_scaled)) * 100,
    accuracy_score(y_test, stacking_classifier.predict(X_test_scaled)) * 100 
]

# Plot model accuracies as percentages
plt.figure(figsize=(10, 6))
plt.bar(model_names, accuracies)
plt.title('Model Accuracy Comparison (in %)')
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)  
for i, v in enumerate(accuracies):
    plt.text(i, v - 2, f'{v:.2f}%', ha='center', va='bottom', fontsize=10) 
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Discussion and Conclusion

Based on the results, it is clear that Stacking RF and other boosting techinques provides a better accuracy than standard Random Forest classifiers. However, it also points to the *importance of preprocessing* the data to achieve higher levels of discrimination between different outcomes. Without enough preprocessing of the data, models tend to learn less and end up relying on majority votes to predict outcomes.

## References

Kaina Zhao and Zhiping Wang. 2024. Research on Diabetes Prediction Based on Machine Learning. In Proceedings of the 6th International Conference on Machine Learning and Machine Intelligence (MLMI '23). Association for Computing Machinery, New York, NY, USA, 29–33. https://doi-org.colorado.idm.oclc.org/10.1145/3635638.3635643

Xi Li, Michele Curiger, Rolf Dornberger, and Thomas Hanne. 2023. Optimized Computational Diabetes Prediction with Feature Selection Algorithms. In Proceedings of the 2023 7th International Conference on Intelligent Systems, Metaheuristics & Swarm Intelligence (ISMSI '23). Association for Computing Machinery, New York, NY, USA, 36–43. https://doi-org.colorado.idm.oclc.org/10.1145/3596947.3596948

Tianqi Chen and Carlos Guestrin. 2016. XGBoost: A Scalable Tree Boosting System. In Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD '16). Association for Computing Machinery, New York, NY, USA, 785–794. https://doi-org.colorado.idm.oclc.org/10.1145/2939672.2939785